# Artificial Neural Network

In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from common import *
import tarfile
import glob
import tensorflow as tf
from keras.backend import clear_session

2022-04-20 20:21:41.957267: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-20 20:21:41.957292: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
if not os.path.exists('aclImdb'):
    if not os.path.exists('aclImdb_v1.tar.gz'):
        files = download_data()
    else:
        files = ['aclImdb_v1.tar.gz']
    
    tarfile.open(files[0]).extractall()

paths = glob.glob('aclImdb/**/pos/*.txt', recursive=True)
pos_frame = get_data(paths, 1)
paths = glob.glob('aclImdb/**/neg/*.txt', recursive=True)
neg_frame = get_data(paths, 0)

df = pd.concat([pos_frame, neg_frame])
print(f'Size of dataset: {df.size}')
df.head()

Size of dataset: 100000


,text,label
0,Previous reviewer Claudio Carvalho gave a much...,1
1,"CONTAINS ""SPOILER"" INFORMATION. Watch this dir...",1
2,This is my first Deepa Mehta film. I saw the f...,1
3,This was a great film in every sense of the wo...,1
4,"A stunningly well-made film, with exceptional ...",1


In [3]:
df = df.sample(n=NUM_SAMPLES, random_state=RANDOM_SEED)
df.head()

,text,label
1247,I just don't get these reviews! I can't help t...,0
10067,Foolish hikers go camping in the Utah mountain...,0
9590,This movie is pretty predictable nuff said.......,0
16668,A convict serving time comes forward to give t...,1
12196,This is a love story set against the back drop...,1


In [4]:
countVectorizer = CountVectorizer()
embeddings = preprocess_data(df['text'], countVectorizer)

X_tr, X_ts, y_tr, y_ts= train_test_split(embeddings, df['label'], test_size=TEST_SIZE, random_state=RANDOM_SEED)

X_tr = X_tr.toarray()
X_ts = X_ts.toarray()

feature_size = X_tr.shape[1]

print(f'Train shape: {X_tr.shape}')
print(f'Test shape: {X_ts.shape}')

Train shape: (2500, 51703)
Test shape: (2500, 51703)


In [5]:
#first basic ann
from keras.models import Sequential
from keras import layers

input_dim = X_tr.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                517040    
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 517,051
Trainable params: 517,051
Non-trainable params: 0
_________________________________________________________________


2022-04-20 20:21:49.207896: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-20 20:21:49.213369: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-20 20:21:49.213439: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2022-04-20 20:21:49.223866: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
history = model.fit(
    X_tr, y_tr,
    epochs=EPOCHS,
    verbose=True,
    validation_data=(X_ts, y_ts),
    batch_size=10
)

2022-04-20 20:21:49.391510: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1034060000 exceeds 10% of free system memory.


Epoch 1/30
247/250 [============================>.] - ETA: 0s - loss: 0.6340 - accuracy: 0.6810

2022-04-20 20:21:53.172309: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1034060000 exceeds 10% of free system memory.


250/250 [==============================] - 6s 21ms/step - loss: 0.6331 - accuracy: 0.6828 - val_loss: 0.5737 - val_accuracy: 0.8372
Epoch 2/30
250/250 [==============================] - 2s 9ms/step - loss: 0.3862 - accuracy: 0.8916 - val_loss: 0.4065 - val_accuracy: 0.8304
Epoch 3/30
250/250 [==============================] - 2s 9ms/step - loss: 0.1261 - accuracy: 0.9768 - val_loss: 0.3879 - val_accuracy: 0.8432
Epoch 4/30
250/250 [==============================] - 2s 8ms/step - loss: 0.0408 - accuracy: 0.9964 - val_loss: 0.3514 - val_accuracy: 0.8640
Epoch 5/30
250/250 [==============================] - 2s 8ms/step - loss: 0.0172 - accuracy: 0.9996 - val_loss: 0.3831 - val_accuracy: 0.8584
Epoch 6/30
250/250 [==============================] - 2s 10ms/step - loss: 0.0090 - accuracy: 1.0000 - val_loss: 0.4026 - val_accuracy: 0.8588
Epoch 7/30
250/250 [==============================] - 2s 9ms/step - loss: 0.0053 - accuracy: 1.0000 - val_loss: 0.4025 - val_accuracy: 0.8608
Epoch 8/30
250/

In [7]:
loss, accuracy = model.evaluate(X_tr, y_tr, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_ts, y_ts, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

import matplotlib.pyplot as plt
plt.style.use('ggplot')

2022-04-20 20:23:08.880857: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1034060000 exceeds 10% of free system memory.


Training Accuracy: 1.0000


2022-04-20 20:23:12.791370: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1034060000 exceeds 10% of free system memory.


Testing Accuracy:  0.8512


# CNN

In [8]:
sentences_train, sentences_test, y_tr, y_ts= train_test_split(df['text'], df['label'], test_size=TEST_SIZE, random_state=RANDOM_SEED)
print(sentences_train)

5677     Honestly, buying this movie was a waste of mon...
8687     When I first watched this film, I thought that...
1221     as a retired USAF MSG (aircraft maint. spec), ...
8034     I was very happy and at the same time quite su...
20403    This movie is told through the eyes of a young...
                               ...                        
20659    Most book adaptations are bad but this film le...
5543     I really felt the movie was ahead of its time....
23070    Movie industry is tricky business - because de...
1304     The tagline for this show is, "He's speaking h...
8938     Kairo, or Pulse as it's known amongst English ...
Name: text, Length: 2500, dtype: object


In [9]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

X_tr = tokenizer.texts_to_sequences(sentences_train)
X_ts = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

In [10]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

X_tr = pad_sequences(X_tr, padding='post', maxlen=maxlen)
X_ts = pad_sequences(X_ts, padding='post', maxlen=maxlen)

print(X_tr[0, :])


[1324 1779   10   15   13    2  398    4  247   40   28    4    1   87
  385   99   11   25  107    8   56  109    1   62  171   11  372    6
    1  272   18   40   35  602   12   21   55    1  777  272   65  612
   10   15    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [11]:
from keras.layers import Embedding
embedding_dim = 50

model_cnn = Sequential()
model_cnn.add(Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model_cnn.add(layers.GlobalMaxPool1D())
model_cnn.add(layers.Dense(10, activation='relu'))
model_cnn.add(layers.Dense(1, activation='sigmoid'))
model_cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           1554450   
                                                                 
 global_max_pooling1d (Globa  (None, 50)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,554,971
Trainable params: 1,554,971
Non-trainable params: 0
_________________________________________________________________


In [12]:
model_cnn.fit(X_tr, y_tr, validation_data=(X_ts, y_ts), epochs=EPOCHS, batch_size=10, verbose=1)

Epoch 1/30
250/250 [==============================] - 6s 21ms/step - loss: 0.6881 - accuracy: 0.5636 - val_loss: 0.6766 - val_accuracy: 0.6512
Epoch 2/30
250/250 [==============================] - 5s 20ms/step - loss: 0.6058 - accuracy: 0.7624 - val_loss: 0.5381 - val_accuracy: 0.7880
Epoch 3/30
250/250 [==============================] - 5s 21ms/step - loss: 0.3849 - accuracy: 0.8744 - val_loss: 0.4317 - val_accuracy: 0.8060
Epoch 4/30
250/250 [==============================] - 5s 20ms/step - loss: 0.2308 - accuracy: 0.9316 - val_loss: 0.4117 - val_accuracy: 0.8080
Epoch 5/30
250/250 [==============================] - 5s 20ms/step - loss: 0.1330 - accuracy: 0.9632 - val_loss: 0.4206 - val_accuracy: 0.8104
Epoch 6/30
250/250 [==============================] - 5s 21ms/step - loss: 0.0701 - accuracy: 0.9860 - val_loss: 0.4445 - val_accuracy: 0.8080
Epoch 7/30
250/250 [==============================] - 5s 20ms/step - loss: 0.0356 - accuracy: 0.9972 - val_loss: 0.4651 - val_accuracy: 0.8068

In [13]:
loss, accuracy = model_cnn.evaluate(X_tr, y_tr, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_cnn.evaluate(X_ts, y_ts, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 1.0000
Testing Accuracy:  0.8044


# BNN

In [14]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
from typing import *
# from sentence_transformers import SentenceTransformer

Prior and posterior likelihoods modeled by a zero-mean, unit-variance Gaussian normal distribution and multivariate Gaussian distribution with learnable mean and variances, respectively.

In [15]:
# zero mean, unit variance multivariate normal
def prior(kernel_size, bias_size, dtype=None):
        n = kernel_size + bias_size
        prior_model = keras.Sequential(
            [
                tfp.layers.DistributionLambda(
                    lambda t: tfp.distributions.MultivariateNormalDiag(
                        loc=tf.zeros(n), scale_diag=tf.ones(n)
                    )
                )
            ]
        )
        return prior_model


# variational multivariate normal (learnable means and variances)
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n),
                dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n)
        ]
    )
    return posterior_model

Create and compile model. The code below creates a hybrid BNN with 2 8-unit deterministic layers and a 2-unit probabilistic layer. The output is a normal distribution based on the parameters learned in the posterior functions/distribution.

In [16]:
inputs = keras.Input(
    shape=(maxlen,), dtype=tf.float64
)

features = keras.layers.BatchNormalization()(inputs)
features = keras.layers.Dense(8, activation='sigmoid')(features)
features = keras.layers.Dense(8, activation='sigmoid')(features)

distribution_params = tfp.layers.DenseVariational(
    units=2,
    make_prior_fn=prior,
    make_posterior_fn=posterior,
    kl_weight=1/TRAIN_SIZE
)(features)

outputs = tfp.layers.IndependentNormal(1)(distribution_params)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()]
)

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


2022-04-20 20:25:52.896910: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Train and evaluate the model on the training and testing data.

In [17]:
model.fit(
    X_tr,
    y_tr,
    epochs=EPOCHS,
    verbose=1,
    validation_data=tuple([X_ts, y_ts])
)

Epoch 1/30
79/79 [==============================] - 2s 9ms/step - loss: 7.7032e-04 - categorical_accuracy: 1.0000 - val_loss: 6.4128e-04 - val_categorical_accuracy: 1.0000
Epoch 2/30
79/79 [==============================] - 0s 3ms/step - loss: 6.6049e-04 - categorical_accuracy: 1.0000 - val_loss: 4.5872e-04 - val_categorical_accuracy: 1.0000
Epoch 3/30
79/79 [==============================] - 0s 3ms/step - loss: 4.5444e-04 - categorical_accuracy: 1.0000 - val_loss: 4.1834e-04 - val_categorical_accuracy: 1.0000
Epoch 4/30
79/79 [==============================] - 0s 4ms/step - loss: 4.5169e-04 - categorical_accuracy: 1.0000 - val_loss: 3.7436e-04 - val_categorical_accuracy: 1.0000
Epoch 5/30
79/79 [==============================] - 0s 6ms/step - loss: 3.4373e-04 - categorical_accuracy: 1.0000 - val_loss: 1.9970e-04 - val_categorical_accuracy: 1.0000
Epoch 6/30
79/79 [==============================] - 0s 4ms/step - loss: 3.2304e-04 - categorical_accuracy: 1.0000 - val_loss: 2.1839e-04 - v

In [18]:
clear_session()

loss, accuracy = model.evaluate(X_tr, y_tr)
print(f'Training accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_ts, y_ts)
print(f'Testing accuracy: {accuracy}')

79/79 [==============================] - 0s 2ms/step - loss: 3.9333e-05 - categorical_accuracy: 1.0000
Training accuracy: 1.0
79/79 [==============================] - 0s 2ms/step - loss: 1.5225e-05 - categorical_accuracy: 1.0000
Testing accuracy: 1.0


In [19]:
PREDICTION_ITERATIONS = 100

results = {
    'Mean': [],
    'Standard Deviation': [],
    'Label': [],
}

# predict on test samples
for i, sample in enumerate(X_ts[0:50, :]):
    sample = sample.reshape(1, sample.shape[0])
    predictions = []

    for _ in range(PREDICTION_ITERATIONS):
        predictions.append(model.predict(sample))

    predictions = np.concatenate(predictions, axis=1)

    results['Mean'].append(np.average(predictions, axis=1)[0])
    results['Standard Deviation'].append(np.std(predictions, axis=1)[0])
    results['Label'].append(y_ts.array[i])

results = pd.DataFrame(data=results)
results.head()

,Mean,Standard Deviation,Label
0,-0.127760,2.224237,1
1,-0.011206,2.037277,0
2,0.451542,2.138670,0
3,0.174447,2.259192,1
4,-0.321685,2.312040,0
